In [1]:
import sys
import warnings
import re

warnings.simplefilter(action="ignore", category=FutureWarning)
warnings.simplefilter(action="ignore", category=RuntimeWarning)

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import GroupKFold
from sklearn.metrics import (
    mean_absolute_percentage_error,
    mean_squared_error,
    d2_tweedie_score,
)
from xgboost import XGBRegressor
from catboost import CatBoostRegressor
from lightgbm import LGBMRegressor
from sklearn.ensemble import RandomForestRegressor, HistGradientBoostingRegressor
from sklearn.preprocessing import OneHotEncoder

from category_encoders import LeaveOneOutEncoder, TargetEncoder

DATA_PATH = "../data"

In [2]:
# Load excel file
df = pd.read_excel(DATA_PATH + "/consumo_material_clean.xlsx")

## Preprocessing

In [3]:
# Separate code into two columns
new_columns = df["P_code"].str.extract(r"([a-zA-Z]+)([0-9]+)", expand=False)
df["P_code_CLASS"] = new_columns[0]
df["P_code_NUM"] = new_columns[1]
df.drop(columns=["P_code"], inplace=True)

In [4]:
# P_date to datetime in day/month/year format
df["P_date"] = pd.to_datetime(df["P_date"], dayfirst=True)
df.sort_values(by=["P_date"], inplace=True)
df.reset_index(drop=True, inplace=True)

C:\Users\AMAR2\AppData\Local\Temp\ipykernel_48196\3581088518.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df["P_date"] = pd.to_datetime(df["P_date"], dayfirst=True)


In [5]:
# separate H_code in three columns by '-'
origin_separated_columns = df["H_code"].str.split("-", expand=True)
df["PURCHASING_HOSPITAL"] = origin_separated_columns[1]
df["PURCHASING_DEPARTMENT"] = origin_separated_columns[2]
df.drop(columns=["H_code"], inplace=True)

In [6]:
# drop duplicates
df.drop_duplicates(inplace=True)

In [7]:
df.columns

Index(['P_date', 'N_Order', 'R_no', 'N_product_purchased', 'N_units', 'Cost',
       'Total_product_purchase', 'Type_purchase', 'Type_logistic', 'PRODUCTO',
       'P_code_CLASS', 'P_code_NUM', 'PURCHASING_HOSPITAL',
       'PURCHASING_DEPARTMENT'],
      dtype='object')

In [8]:
# basic date features
def generate_date_features(df):
    df["YEAR"] = df["P_date"].dt.year
    df["MONTH"] = np.sin(2 * np.pi * df["P_date"].dt.month / 12)
    df["DAYOFMONTH"] = np.sin(2 * np.pi * df["P_date"].dt.day / 31)
    df["DAYOFYEAR"] = np.sin(2 * np.pi * df["P_date"].dt.dayofyear / 365)
    return df

In [9]:
def add_timeseries_features(df):
    # MEANS
    df["ROLLING_MEAN_3M"] = df["N_product_purchased"].rolling(90).mean()
    df["WEIGHTED_MEAN_3M"] = (
        df["N_product_purchased"]
        .rolling(90)
        .apply(lambda x: np.average(x, weights=range(1, len(x) + 1)))
    )
    df["ROLLING_MEAN_1Y"] = df["N_product_purchased"].rolling(365).mean()
    df["WEIGHTED_MEAN_1Y"] = (
        df["N_product_purchased"]
        .rolling(365)
        .apply(lambda x: np.average(x, weights=range(1, len(x) + 1)))
    )
    df["EWMA_3M"] = df["N_product_purchased"].ewm(span=90).mean()
    df["EWMA_1Y"] = df["N_product_purchased"].ewm(span=365).mean()

    # LAGS
    df["SHIFT_1W"] = df["N_product_purchased"].shift(7)
    df["SHIFT_2W"] = df["N_product_purchased"].shift(14)
    df["SHIFT_1M"] = df["N_product_purchased"].shift(30)
    df["SHIFT_3M"] = df["N_product_purchased"].shift(90)
    df["SHIFT_1Y"] = df["N_product_purchased"].shift(365)

    return df

In [10]:
def generate_train_test_df(full_df):
    # Get train and test sets
    train = full_df[full_df["YEAR"] < 2023]
    X_train = train.drop(columns=["N_product_purchased", "P_date"])
    y_train = train["N_product_purchased"]

    test = full_df[full_df["YEAR"] == 2023]
    X_test = test.drop(columns=["N_product_purchased", "P_date"])
    y_test = test["N_product_purchased"]

    return train, X_train, y_train, test, X_test, y_test

In [11]:
def smape_score(A, F):
    return 100 / len(A) * np.sum(2 * np.abs(F - A) / (np.abs(A) + np.abs(F)))

In [12]:
def train_model_eval(X_train, y_train, X_test, y_test):
    model = XGBRegressor(random_state=42, n_estimators=1000)
    model.fit(X_train, y_train)

    y_test_pred = model.predict(X_test)
    y_test_pred = np.maximum(y_test_pred, 0)  # clip negative predictions to 0

    test_loss = mean_squared_error(y_test, y_test_pred, squared=False)
    mape_error = mean_absolute_percentage_error(y_test, y_test_pred)
    tweedie = d2_tweedie_score(y_test, y_test_pred)
    smape_err = smape_score(y_test, y_test_pred)

    return test_loss, mape_error, smape_err, tweedie

In [14]:
# iterate over products
columns = [
    "P_date",
    "N_product_purchased",
    "PURCHASING_HOSPITAL",
    "PURCHASING_DEPARTMENT",
]

product_losses = pd.DataFrame(columns=["PRODUCT", "Tweedie", "MSE", "SMAPE"])
for product in df["P_code_NUM"].unique():
    partial_df = df[df["P_code_NUM"] == product]
    partial_df = partial_df.groupby(columns).sum().reset_index()

    loo = LeaveOneOutEncoder()
    partial_df["PURCHASING_HOSPITAL"] = loo.fit_transform(
        partial_df["PURCHASING_HOSPITAL"], partial_df["N_product_purchased"]
    )

    loo = TargetEncoder()
    partial_df["PURCHASING_DEPARTMENT"] = loo.fit_transform(
        partial_df["PURCHASING_DEPARTMENT"], partial_df["N_product_purchased"]
    )

    partial_df = partial_df[columns]
    partial_df = generate_date_features(partial_df)
    partial_df = add_timeseries_features(partial_df)

    is_2023_in_df = 2023 in partial_df["YEAR"].unique()
    product_blacklist = ["85758", "73753"]  # stops selling on 2023
    if not is_2023_in_df or product in product_blacklist:
        continue

    train, X_train, y_train, test, X_test, y_test = generate_train_test_df(partial_df)
    test_loss, mape_error, smape, tweedie = train_model_eval(
        X_train, y_train, X_test, y_test
    )

    product_losses = pd.concat(
        [
            product_losses,
            pd.DataFrame(
                [[product, tweedie, test_loss, smape]],
                columns=["PRODUCT", "Tweedie", "MSE", "SMAPE"],
            ),
        ]
    )

In [15]:
product_losses.sort_values(by=["SMAPE"], ascending=False)

,PRODUCT,Tweedie,MSE,SMAPE
0,41691,-4.301985,214.036913,81.928501
0,64663,-0.753391,374.491743,60.109700
0,65056,0.217848,417.637705,60.057900
0,65007,-1.666662,81.009185,58.333206
0,64544,-0.426951,360.926398,46.807484
0,64932,-1.205931,399.675414,46.407764
0,46846,-2.521774,6.138824,45.650925
0,65894,0.259914,251.919529,45.617468
0,67835,-4.384140,154.691662,43.115805
0,64488,0.092849,255.990562,42.208447


In [16]:
product_losses["SMAPE"].mean(), product_losses["MSE"].mean(), product_losses[
    product_losses["Tweedie"] != -np.inf
]["Tweedie"].mean()

(27.734995180302565, 212.9172821385805, -0.4207011055870864)

28.576833408153984